In [3]:
import pandas as pd
import numpy as np

from pathlib import Path

from gensim.models import word2vec, KeyedVectors
from tqdm import tqdm
tqdm.pandas()

/Users/shinichiro.saito/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
sub = pd.read_csv('../../data/atmacup10__sample_submission.csv')
color = pd.read_csv('../../data/color.csv')
historical_person = pd.read_csv('../../data/historical_person.csv')
maker = pd.read_csv('../../data/maker.csv')
material = pd.read_csv('../../data/material.csv')
collection = pd.read_csv('../../data/object_collection.csv')
palette = pd.read_csv('../../data/palette.csv')
principal_maker_occupation = pd.read_csv('../../data/principal_maker_occupation.csv')
principal_maker = pd.read_csv('../../data/principal_maker.csv')

principal_maker_role = pd.merge(principal_maker,principal_maker_occupation,on='id',how='left') 

#production_place = pd.read_csv('../../data/production_place.csv')
production = pd.read_csv('../../extract_data/production_place_rename.csv').iloc[:,1:][['object_id','name']]
production_country = pd.read_csv('../../extract_data/production_place_rename.csv').iloc[:,1:][['object_id','country_name']].rename(columns={'country_name':'name'})

technique = pd.read_csv('../../data/technique.csv')
test = pd.read_csv('../../data/test.csv')
train = pd.read_csv('../../data/train.csv')

In [3]:
principal_maker_role

,id,object_id,qualification,roles,productionPlaces,maker_name,name
0,1,000405d9a5e3f49fc49d,mentioned on object,NaN,NaN,Bernardus Bruining,NaN
1,2,001020bd00b149970f78,workshop of,painter,NaN,Jan Antonisz van Ravesteyn,draughtsman
2,2,001020bd00b149970f78,workshop of,painter,NaN,Jan Antonisz van Ravesteyn,painter
3,3,0011d6be41612ec9eae3,NaN,painter,NaN,Jan Hackaert,print maker
4,3,0011d6be41612ec9eae3,NaN,painter,NaN,Jan Hackaert,draughtsman
...,...,...,...,...,...,...,...
36183,24534,fff4bbb55fd7702d294e,NaN,NaN,NaN,Henry W. Taunt,NaN
36184,24535,fffbe07b997bec00e203,attributed to,NaN,Great Britain,John Jabez Edwin Mayall,NaN
36185,24536,fffd1675758205748d7f,NaN,NaN,NaN,Francis Frith,publisher
36186,24537,fffd43b134ba7197d890,NaN,NaN,NaN,Henry W. Taunt,NaN


In [31]:
qual = principal_maker_role[['object_id','qualification']]
role = principal_maker_role[['object_id','roles']]
name = principal_maker_role[['object_id','name']]

In [42]:
qual = qual.rename(columns={'qualification':'name'})
role = role.rename(columns={'roles':'name'})

In [49]:
qual = qual.fillna('missing')
role = role.fillna('missing')
name = name.fillna('missing')

In [50]:
qual['name'].nunique()

17

In [51]:
role['name'].nunique()

12

In [52]:
name['name'].nunique()

13

In [53]:
qual_role = pd.concat([qual, role], axis=0).reset_index(drop=True)
qual_name = pd.concat([qual, name], axis=0).reset_index(drop=True)
role_name = pd.concat([role, name], axis=0).reset_index(drop=True)
qual_role_name = pd.concat([qual, role, name], axis=0).reset_index(drop=True)

In [54]:
# 単語ベクトル表現の次元数
# 元の語彙数をベースに適当に決めました
# 大体1/3ぐらい？
model_size = {
    "qualification": 5,
    "roles": 4,
    "name": 4,
    "qualification_roles" : 20,
    "qualification_name": 20,
    "roles_name": 20,
    "qualification_roles_name" : 30,
}

n_iter = 100

In [55]:
w2v_dfs = []
for df, df_name in zip(
        [
            qual, role, name, 
            qual_role, qual_name, role_name,
            qual_role_name
        ], [
            "qualification", "roles", "name",
            "qualification_roles", "qualification_name", "roles_name",
            "qualification_roles_name"
        ]):
    df_group = df.groupby("object_id")["name"].apply(list).reset_index()
    # Word2Vecの学習
    w2v_model = word2vec.Word2Vec(df_group["name"].values.tolist(),
                                  size=model_size[df_name],
                                  min_count=1,
                                  window=1,
                                  iter=n_iter)

    # 各文章ごとにそれぞれの単語をベクトル表現に直し、平均をとって文章ベクトルにする
    sentence_vectors = df_group["name"].progress_apply(
        lambda x: np.mean([w2v_model.wv[e] for e in x], axis=0))
    sentence_vectors = np.vstack([x for x in sentence_vectors])
    sentence_vector_df = pd.DataFrame(sentence_vectors,
                                      columns=[f"{df_name}_w2v_{i}"
                                               for i in range(model_size[df_name])])
    sentence_vector_df.index = df_group["object_id"]
    w2v_dfs.append(sentence_vector_df)

100%|██████████| 24034/24034 [00:00<00:00, 47669.87it/s]


In [56]:
len(w2v_dfs)

7

In [72]:
w2v_dfs[0]

,qualification_w2v_0,qualification_w2v_1,qualification_w2v_2,qualification_w2v_3,qualification_w2v_4
object_id,,,,,
000405d9a5e3f49fc49d,-1.407384,-0.298591,-0.139830,-2.478325,-2.549996
001020bd00b149970f78,-2.742899,0.964579,-3.054511,-0.525887,-0.858646
0011d6be41612ec9eae3,-1.417162,2.441867,-0.156843,-0.055526,1.289634
0012765f7a97ccc3e9e9,-1.025733,1.105202,1.408582,1.419923,-0.180969
00133be3ff222c9b74b0,-1.417162,2.441867,-0.156843,-0.055526,1.289634
...,...,...,...,...,...
fff4bbb55fd7702d294e,-1.417162,2.441867,-0.156843,-0.055526,1.289634
fffbe07b997bec00e203,-1.025733,1.105202,1.408582,1.419923,-0.180969
fffd1675758205748d7f,-1.417162,2.441867,-0.156843,-0.055526,1.289634


In [58]:
w2v_dfs[0].to_csv('../qualification_w2v.csv')

In [60]:
w2v_dfs[1].to_csv('../roles_w2v.csv')

In [62]:
w2v_dfs[2].to_csv('../name_w2v_0.csv')

In [64]:
w2v_dfs[3].to_csv('../qualification_roles_w2v.csv')

In [66]:
w2v_dfs[4].to_csv('../qualification_name_w2v.csv')

In [68]:
w2v_dfs[5].to_csv('../roles_name_w2v.csv')

In [70]:
w2v_dfs[6].to_csv('../qualification_roles_name_w2v.csv')